In [1]:
import pandas as pd
import seaborn as sns
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
#import pandas_profiling
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import sklearn.feature_extraction.text as text
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import warnings
!pip install contractions
import contractions
from collections import Counter
!pip install wordcloud
from wordcloud import WordCloud
from sklearn. preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import nltk
nltk.download('stopwords')
import numpy as np
warnings.filterwarnings('ignore')
import gensim
!pip install imblearn
from imblearn.pipeline import make_pipeline
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gruhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gruhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = pd.read_json('data.json',lines = True)

In [3]:
data['category'].unique()

array(['U.S. NEWS', 'COMEDY', 'PARENTING', 'WORLD NEWS', 'CULTURE & ARTS',
       'TECH', 'SPORTS', 'ENTERTAINMENT', 'POLITICS', 'WEIRD NEWS',
       'ENVIRONMENT', 'EDUCATION', 'CRIME', 'SCIENCE', 'WELLNESS',
       'BUSINESS', 'STYLE & BEAUTY', 'FOOD & DRINK', 'MEDIA',
       'QUEER VOICES', 'HOME & LIVING', 'WOMEN', 'BLACK VOICES', 'TRAVEL',
       'MONEY', 'RELIGION', 'LATINO VOICES', 'IMPACT', 'WEDDINGS',
       'COLLEGE', 'PARENTS', 'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE',
       'HEALTHY LIVING', 'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST',
       'FIFTY', 'ARTS', 'DIVORCE'], dtype=object)

In [4]:
data.drop_duplicates(inplace = True)

In [5]:
data.drop_duplicates(subset=['headline', 'short_description'], inplace = True)

In [6]:
data.shape

(209038, 6)

In [7]:
data.isnull().values.any()

False

In [8]:
df = data

In [9]:
df['all_words'] = df['headline']+" "+df['short_description']

In [10]:
df['all_words'][0]

'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.'

# data Cleaning / preprocessing

In [11]:
def expand_contractions(combined_words):
    expanded_words = []   
    for word in combined_words.split():
        expanded_words.append(contractions.fix(word))  
    expanded_text = ' '.join(expanded_words)
    return expanded_text

In [12]:
stop_words = set(stopwords.words('english'))

In [13]:
def remove_stop_words(x):
    lst = [i for i in x.split() if not i in stop_words]
    data = ' '.join(lst)
    return data

In [14]:
def clean_data(data):
    data = str(data)
    data = data.lower()
    data = expand_contractions(data)
    data = re.sub('@[A-Za-z0-9_]+',' ', data)
    data = re.sub('#[A-Za-z0-9_]+',' ', data)
    data = re.sub('(http\S+)', ' ', data)
    data = re.sub('www.\S+', ' ', data)
    data = re.sub('[()!?]', ' ', data)
    data = re.sub('\[.*?\]',' ', data)
    data = re.sub('[^a-z0-9]',' ', data)
    data = re.sub(' +',' ', data)
    data = remove_stop_words(data)
    return data

In [15]:
clean_df = df.loc[:,['all_words','category']]

In [16]:
clean_df['clean_all_words'] = clean_df['all_words'].apply(clean_data)

In [17]:
clean_df.head()

,all_words,category,clean_all_words
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,4 million americans roll sleeves omicron targe...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,american airlines flyer charged banned life pu...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,23 funniest tweets cats dogs week sept 17 23 d...
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,funniest tweets parents week sept 17 23 accide...
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,woman called cops black bird watcher loses law...


In [18]:
X = clean_df['clean_all_words'].tolist()
label = LabelEncoder()
Y = label.fit_transform(clean_df['category']).tolist()

# Test Train With SMOTE

In [19]:
x_train,x_test, y_train,y_test = train_test_split(X,Y)

In [20]:
from imblearn.over_sampling import SMOTE
vectorizer = TfidfVectorizer()
x_train_ex = vectorizer.fit_transform(x_train)
x_test_ex = vectorizer.transform(x_test)

smote = SMOTE(random_state=42)
x_train_smote, y_train_smote = smote.fit_resample(x_train_ex, y_train)

In [21]:
x_train_smote.shape

(1123458, 77454)

In [22]:
x_train_ex.shape

(156778, 77454)

# Classification

In [23]:

from sklearn.naive_bayes import MultinomialNB

pipeline = make_pipeline(
    TfidfVectorizer(),
    SMOTE(random_state=42),
    MultinomialNB()
)
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

accuracy 0.5530424799081516
              precision    recall  f1-score   support

           0    0.23702   0.28378   0.25830       370
           1    0.22611   0.30990   0.26146       313
           2    0.42105   0.44991   0.43500      1138
           3    0.47941   0.43467   0.45594      1500
           4    0.38133   0.45981   0.41691       311
           5    0.52332   0.41679   0.46402      1346
           6    0.44774   0.68326   0.54098       884
           7    0.25672   0.32950   0.28859       261
           8    0.59535   0.73818   0.65911       867
           9    0.26471   0.54435   0.35620       248
          10    0.74155   0.57111   0.64526      4416
          11    0.32272   0.39583   0.35556       384
          12    0.18038   0.33784   0.23518       370
          13    0.63221   0.63659   0.63439      1585
          14    0.24308   0.37386   0.29461       329
          15    0.33373   0.43662   0.37831       639
          16    0.46698   0.11633   0.18627      1702

In [24]:
from sklearn.linear_model import LogisticRegression

pipeline = make_pipeline(
    TfidfVectorizer(),
    SMOTE(random_state=42),
    LogisticRegression(n_jobs=1, C=1e5)
)
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

accuracy 0.5436471488710295
              precision    recall  f1-score   support

           0    0.24891   0.30811   0.27536       370
           1    0.24934   0.30351   0.27378       313
           2    0.39330   0.47452   0.43011      1138
           3    0.43462   0.47200   0.45254      1500
           4    0.31905   0.43087   0.36662       311
           5    0.37296   0.45914   0.41159      1346
           6    0.44779   0.53846   0.48896       884
           7    0.34653   0.26820   0.30238       261
           8    0.67269   0.73010   0.70022       867
           9    0.29000   0.46774   0.35802       248
          10    0.68314   0.59171   0.63415      4416
          11    0.37811   0.39583   0.38677       384
          12    0.17724   0.29459   0.22132       370
          13    0.63625   0.62902   0.63261      1585
          14    0.22439   0.27964   0.24899       329
          15    0.28846   0.37559   0.32631       639
          16    0.30364   0.33843   0.32009      1702

In [25]:
from sklearn.linear_model import SGDClassifier

pipeline = make_pipeline(
    TfidfVectorizer(),
    SMOTE(random_state=42),
    SGDClassifier(loss='hinge', penalty='l2')
)
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

accuracy 0.4666666666666667
              precision    recall  f1-score   support

           0    0.16168   0.29189   0.20809       370
           1    0.14470   0.32268   0.19980       313
           2    0.37645   0.45518   0.41209      1138
           3    0.46799   0.34600   0.39785      1500
           4    0.21861   0.62701   0.32419       311
           5    0.47855   0.43091   0.45348      1346
           6    0.42633   0.61538   0.50370       884
           7    0.18739   0.42146   0.25943       261
           8    0.59410   0.79008   0.67822       867
           9    0.16583   0.66532   0.26549       248
          10    0.78217   0.30005   0.43372      4416
          11    0.24526   0.47135   0.32264       384
          12    0.16438   0.38919   0.23114       370
          13    0.62190   0.53754   0.57665      1585
          14    0.16692   0.33739   0.22334       329
          15    0.28655   0.38341   0.32798       639
          16    0.29388   0.08461   0.13139      1702

# Test Train With Random Over Sampler

In [26]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import MultinomialNB

pipeline = make_pipeline(
    TfidfVectorizer(),
    RandomOverSampler(random_state=42),
    MultinomialNB()
)
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

accuracy 0.5465939533103712
              precision    recall  f1-score   support

           0    0.23326   0.27297   0.25156       370
           1    0.21739   0.28754   0.24759       313
           2    0.42079   0.44112   0.43072      1138
           3    0.48232   0.42733   0.45316      1500
           4    0.35610   0.46945   0.40499       311
           5    0.46510   0.48514   0.47491      1346
           6    0.43228   0.69683   0.53356       884
           7    0.24699   0.31418   0.27656       261
           8    0.56836   0.73356   0.64048       867
           9    0.24501   0.54435   0.33792       248
          10    0.74977   0.55639   0.63876      4416
          11    0.32573   0.40885   0.36259       384
          12    0.15838   0.32703   0.21340       370
          13    0.62841   0.63912   0.63372      1585
          14    0.24468   0.34954   0.28786       329
          15    0.33169   0.42097   0.37103       639
          16    0.33076   0.25206   0.28610      1702

In [27]:
from sklearn.linear_model import LogisticRegression

pipeline = make_pipeline(
    TfidfVectorizer(),
    RandomOverSampler(random_state=42),
    LogisticRegression(n_jobs=1, C=1e5)
)
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

accuracy 0.5462686567164179
              precision    recall  f1-score   support

           0    0.27402   0.20811   0.23656       370
           1    0.25267   0.22684   0.23906       313
           2    0.40198   0.49912   0.44532      1138
           3    0.40090   0.47467   0.43468      1500
           4    0.41538   0.34727   0.37828       311
           5    0.38561   0.48960   0.43142      1346
           6    0.45438   0.55204   0.49847       884
           7    0.31439   0.31801   0.31619       261
           8    0.67891   0.74625   0.71099       867
           9    0.31173   0.40726   0.35315       248
          10    0.67674   0.60824   0.64067      4416
          11    0.38585   0.31250   0.34532       384
          12    0.28165   0.24054   0.25948       370
          13    0.62151   0.64543   0.63324      1585
          14    0.26603   0.25228   0.25897       329
          15    0.32210   0.40376   0.35833       639
          16    0.27502   0.33255   0.30106      1702

In [28]:
from sklearn.linear_model import SGDClassifier

pipeline = make_pipeline(
    TfidfVectorizer(),
    RandomOverSampler(random_state=42),
    SGDClassifier(loss='hinge', penalty='l2')
)
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)

print('accuracy %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=5))

accuracy 0.4719862227324914
              precision    recall  f1-score   support

           0    0.18237   0.32432   0.23346       370
           1    0.14663   0.31949   0.20101       313
           2    0.39953   0.44903   0.42284      1138
           3    0.49546   0.32733   0.39422      1500
           4    0.21324   0.65273   0.32146       311
           5    0.44240   0.48217   0.46143      1346
           6    0.42334   0.62783   0.50569       884
           7    0.17966   0.40613   0.24912       261
           8    0.56675   0.81776   0.66950       867
           9    0.16199   0.69758   0.26292       248
          10    0.77734   0.31386   0.44717      4416
          11    0.23545   0.46354   0.31228       384
          12    0.17143   0.40541   0.24096       370
          13    0.59786   0.56278   0.57979      1585
          14    0.15729   0.33131   0.21331       329
          15    0.28781   0.39906   0.33443       639
          16    0.33368   0.18625   0.23906      1702